In [1]:
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy.ma as ma
import numpy as np
import netCDF4 as nc4
import xarray as xr
import sys
sys.path.insert(0,"/home/albert7a/lib/python/pypago")
import os
import matplotlib
import pylab as plt
import numpy as np
import pyproj

%matplotlib inline
%autosave 60

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:16: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


Autosaving every 60 seconds


In [2]:
#data and grid
gridname='/store/molines/NATL60/NATL60-I/NATL60_coordinates_v4.nc'
dgrid=xr.open_dataset(gridname)
navlon=dgrid['nav_lon']
navlat=dgrid['nav_lat']

wname='/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d07.1d_gridW.nc'
dw=xr.open_dataset(wname)
w=dw['vovecrtz']
depthw=dw['depthw']

In [3]:
ee = pyproj.Geod(ellps='GRS80')

In [4]:
def section_from_debut_fin_points(lon_ar7w_debut,lat_ar7w_debut,lon_ar7w_fin,lat_ar7w_fin,navlon,navlat,var):
    
    ee = pyproj.Geod(ellps='GRS80') # ellipsoid used for distance calculation
    
    def distance(lat1,lon1,lat2,lon2,geoid):
        az12,az21,dist = geoid.inv(lon1,lat1,lon2,lat2)
        return dist

    def loclatlon(navlon,navlat,lon,lat):
        ind_debut=np.where((navlon>lon-0.1)&(navlon<lon+0.1)&(navlat>lat-0.1)&(navlat<lat+0.1))
        dis=np.empty(len(ind_debut[0]))
        for ii in range(0,len(ind_debut[0])):
            dis[ii]=np.abs(distance(lat,lon,navlat[ind_debut[0][ii],ind_debut[1][ii]],navlon[ind_debut[0][ii],ind_debut[1][ii]],ee))
        ind=np.argmin(dis)
        i_debut=ind_debut[0][ind]
        j_debut=ind_debut[1][ind]
    
        return i_debut,j_debut
 
    def secfromdebuttofin(i_debut,j_debut,i_fin,j_fin):
        off=1
        fromi=np.atleast_1d(i_debut)+off
        fromj=np.atleast_1d(j_debut)+off
        toi=np.atleast_1d(i_fin)+off
        toj=np.atleast_1d(j_fin)+off

        veci=fromi
        vecj=fromj

        a=(toj-fromj)/((toi-fromi).astype(np.float))
        b=toj-toi*a

        while not((veci[-1]==toi) & (vecj[-1]==toj)):
            newi=veci[-1]+np.sign(toi-fromi)
            newj=vecj[-1]+np.sign(toj-fromj)
            y=a*newi+b
            x=(newj-b)/(a.astype(np.float))

            if ((np.abs(x-veci[-1]))>=(np.abs(y-vecj[-1]))):
                veci=np.append(veci,newi)
                vecj=np.append(vecj,vecj[-1])
            else:
                veci=np.append(veci,veci[-1])
                vecj=np.append(vecj,newj)

        veci=veci.astype(np.int)-off
        vecj=vecj.astype(np.int)-off

        dis=np.empty(len(veci))
        for ii in range(0,len(veci)):
            dis[ii]=np.abs(distance(navlat[veci[0],vecj[0]],navlon[veci[0],vecj[0]],navlat[veci[ii],vecj[ii]],navlon[veci[ii],vecj[ii]],ee))
    
        return veci,vecj,dis
            
    i_debut,j_debut=loclatlon(navlon,navlat,lon_ar7w_debut,lat_ar7w_debut)
    i_fin,j_fin=loclatlon(navlon,navlat,lon_ar7w_fin,lat_ar7w_fin)

    veci,vecj,dis=secfromdebuttofin(i_debut,j_debut,i_fin,j_fin)

    if np.ndim(var) == 4:
        [nt,nk,nj,ni]=var.shape
        var_sec=np.empty([nt,nk,len(veci)])
        for ii in range(0,len(veci)):
            var_sec[:,:,ii]=var[:,:,veci[ii],vecj[ii]]
    if np.ndim(var) == 3:
        [nt,nj,ni]=var.shape
        var_sec=np.empty([nt,len(veci)])
        for ii in range(0,len(veci)):
            var_sec[:,ii]=var[:,veci[ii],vecj[ii]]
    
    return veci,vecj,dis,var_sec

In [5]:
lon_ar7w_debut=np.array(-56).astype(np.float)
lon_ar7w_fin=np.array(-48).astype(np.float)

lat_ar7w_debut=np.array(53.5).astype(np.float)
lat_ar7w_fin=np.array(60.5).astype(np.float)

veci,vecj,dis,wlabrador=section_from_debut_fin_points(lon_ar7w_debut,lat_ar7w_debut,lon_ar7w_fin,lat_ar7w_fin,navlon,navlat,w)


In [ ]:
#Labrador section beginning and end point

lon_ar7w_debut=np.array(-56).astype(np.float)
lon_ar7w_fin=np.array(-48).astype(np.float)

lat_ar7w_debut=np.array(53.5).astype(np.float)
lat_ar7w_fin=np.array(60.5).astype(np.float)

veci,vecj,dis,wlabrador=section_from_debut_fin_points(lon_ar7w_debut,lat_ar7w_debut,lon_ar7w_fin,lat_ar7w_fin,navlon,navlat,w)
wlabrador_smooth=np.ones(wlabrador.shape)
wlabrador_vert_variance=np.ones(wlabrador.shape)

delta=1e-6
for ii in range(len(veci)):
    profil=wlabrador[0,:,ii]
    window_len=3
    s=np.r_[profil[window_len-1:0:-1],profil,profil[-2:-window_len-1:-1]]
    whanning=np.hanning(window_len)
    yhanning=np.convolve(whanning/whanning.sum(),s,mode='valid')
    wlabrador_smooth[0,:,ii]=yhanning[0:300]
    wlabrador_vert_variance[0,:,ii]=np.sqrt(((wlabrador[0,:,ii]-wlabrador_smooth[0,:,ii])**2)/((wlabrador[0,:,ii]+delta)**2))


ma_wlabrador = np.ma.array (wlabrador, mask=np.isnan(wlabrador))
ma_wlabrador_smooth = np.ma.array (wlabrador_smooth, mask=np.isnan(wlabrador_smooth))
ma_wlabrador_vert_variance = np.ma.array (wlabrador_vert_variance, mask=np.isnan(wlabrador_vert_variance))
matplotlib.rcParams.update({'font.size': 22})

plt.figure(figsize=(30,40))

ax=plt.subplot(3,1,1)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*ma_wlabrador[0,:,:],cmap=cmap2,vmin=-10,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m/day')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Canadian land in km')
plt.ylabel('depth in m')
plt.title('Vertical Velocity Labrador cross-section')

ax=plt.subplot(3,1,2)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*(ma_wlabrador[0,:,:]-ma_wlabrador_smooth[0,:,:]),cmap=cmap2,vmin=-10,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m/day')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Canadian land in km')
plt.ylabel('depth in m')
plt.title('De-smoothed Vertical Velocity Labrador cross-section')

ax=plt.subplot(3,1,3)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*np.abs(ma_wlabrador[0,:,:]-ma_wlabrador_smooth[0,:,:]),cmap='YlOrBr',vmin=0,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='rms')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Canadian land in km')
plt.ylabel('depth in m')
plt.title('Vertical variance Labrador cross-section')

In [ ]:
#Ovide section beginning and end point

lon_ovide_debut=np.array(-44).astype(np.float)
lon_ovide_fin=np.array(-10).astype(np.float)

lat_ovide_debut=np.array(60).astype(np.float)
lat_ovide_fin=np.array(53.75).astype(np.float)

veci,vecj,dis,wovide=section_from_debut_fin_points(lon_ovide_debut,lat_ovide_debut,lon_ovide_fin,lat_ovide_fin,navlon,navlat,w)
wovide_smooth=np.ones(wovide.shape)
wovide_vert_variance=np.ones(wovide.shape)
delta=1e-6
for ii in range(len(veci)):
    profil=wovide[0,:,ii]
    window_len=3
    s=np.r_[profil[window_len-1:0:-1],profil,profil[-2:-window_len-1:-1]]
    whanning=np.hanning(window_len)
    yhanning=np.convolve(whanning/whanning.sum(),s,mode='valid')
    wovide_smooth[0,:,ii]=yhanning[0:300]
    wovide_vert_variance[0,:,ii]=np.sqrt(((wovide[0,:,ii]-wovide_smooth[0,:,ii])**2)/((wovide[0,:,ii]+delta)**2))
ma_wovide = np.ma.array (wovide, mask=np.isnan(wovide))
ma_wovide_smooth = np.ma.array (wovide_smooth, mask=np.isnan(wovide_smooth))
ma_wovide_vert_variance = np.ma.array (wovide_vert_variance, mask=np.isnan(wovide_vert_variance))
matplotlib.rcParams.update({'font.size': 22})

plt.figure(figsize=(30,40))

ax=plt.subplot(3,1,1)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*ma_wovide[0,:,:],cmap=cmap2,vmin=-10,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m/day')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Greenland in km')
plt.ylabel('depth in m')
plt.title('Vertical Velocity Ovide cross-section')

ax=plt.subplot(3,1,2)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*(ma_wovide[0,:,:]-ma_wovide_smooth[0,:,:]),cmap=cmap2,vmin=-10,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m/day')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Greenland in km')
plt.ylabel('depth in m')
plt.title('De-smoothed Vertical Velocity Ovide cross-section')

ax=plt.subplot(3,1,3)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*np.abs(ma_wovide[0,:,:]-ma_wovide_smooth[0,:,:]),cmap='YlOrBr',vmin=0,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='rms')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Greenland in km')
plt.ylabel('depth in m')
plt.title('Vertical variance Ovide cross-section')